In [1]:
# =====================================
# Sustainable Supply Chain Decision Tool
# =====================================

# -----------------------------
# 1. Import libraries and module
# -----------------------------
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Add project root to Python path
import sys, os
sys.path.append(os.path.abspath(".."))

from src.decision_matrix import (
    calculate_weighted_score,
    monte_carlo_simulation,
    recommend_suppliers,
    plot_mc_boxplot,
    plot_mc_histogram,
    normalize_min_max
)


In [2]:
# -----------------------------
# 2. Load supplier data
# -----------------------------
df = pd.read_csv("../data/suppliers.csv")  # adjust path
print("Supplier data loaded:")
df.head()


Supplier data loaded:


,SupplierID,Cost_per_unit,Delivery_time_days,Carbon_kgCO2e,QualityRating,ComplianceScore
0,S1,24.98,8,18.69,6.8,79.1
1,S2,48.03,7,28.61,8.1,72.9
2,S3,39.28,4,24.44,8.4,90.5
3,S4,33.95,10,18.11,6.2,83.2
4,S5,16.24,14,32.53,8.4,73.7


In [3]:
# -----------------------------
# 3. Define normalization for metrics
# -----------------------------
normalize_cols = {
    "Cost_per_unit": False,          # lower is better
    "Delivery_time_days": False,     # lower is better
    "Carbon_kgCO2e": False,          # lower is better
    "QualityRating": True,           # higher is better
    "ComplianceScore": True          # higher is better
}

metrics = list(normalize_cols.keys())


In [4]:
print("Choose weight option:")
print("1: Manually enter weights (from weights.csv)")
print("2: Use suggested optimal weights (calculated automatically)")

weight_choice = input("Enter 1 or 2: ").strip()


Choose weight option:
1: Manually enter weights (from weights.csv)
2: Use suggested optimal weights (calculated automatically)


Enter 1 or 2:  2


In [ ]:
if weight_choice == "1":
    # Load weights from CSV
    weights_df = pd.read_csv("../data/weights.csv")
    weights_dynamic = dict(zip(weights_df['Metric'], weights_df['Weight']))
    print("Using manual weights from weights.csv:")
    print(weights_dynamic)
elif weight_choice == "2":
    print("Calculating suggested optimal weights...")

    increments = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]  # grid increments
    possible_weights = [w for w in itertools.product(increments, repeat=len(metrics)) if abs(sum(w)-1.0)<1e-5]

    best_score = -np.inf
    best_weights = None

    for w in possible_weights:
        trial_weights = dict(zip(metrics, w))
        df_weighted = calculate_weighted_score(df.copy(), trial_weights, normalize_cols)
        df_mc, mc_results_trial = monte_carlo_simulation(df_weighted, trial_weights, normalize_cols)
        top_mean = df_mc["MC_MeanScore"].max()
        if top_mean > best_score:
            best_score = top_mean
            best_weights = trial_weights

    weights_dynamic = best_weights
    print("Suggested optimal weights:")
    print(weights_dynamic)


Calculating suggested optimal weights...


In [ ]:
# Weighted scores
df_weighted = calculate_weighted_score(df.copy(), weights_dynamic, normalize_cols)

# Monte Carlo simulation
df_mc, mc_results_dynamic = monte_carlo_simulation(df_weighted, weights_dynamic, normalize_cols)

# Display top 3 suppliers by Monte Carlo mean score
top_3 = recommend_suppliers(df_mc, top_n=3)
print("\nTop 3 Supplier Recommendations:")
top_3


In [ ]:
# Normalize numeric columns for clustering
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[norm_cols := list(normalize_cols.keys())])

# KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df['Cluster'] = kmeans.fit_predict(df_scaled)

print("\nSuppliers with cluster assignment:")
df[['SupplierID', 'Cluster'] + norm_cols]

# Optional: visualize first 2 metrics
plt.figure(figsize=(8,5))
for c in df['Cluster'].unique():
    subset = df[df['Cluster']==c]
    plt.scatter(subset['Cost_per_unit'], subset['Delivery_time_days'], label=f'Cluster {c}')
plt.xlabel('Cost_per_unit')
plt.ylabel('Delivery_time_days')
plt.legend()
plt.title('Supplier Clusters')
plt.show()


In [ ]:
# Boxplot for all suppliers
plot_mc_boxplot(df_mc, mc_results_dynamic)

# Histogram for top supplier
top_idx = df_mc["MC_MeanScore"].idxmax()
plot_mc_histogram(mc_results_dynamic, df_mc, supplier_idx=top_idx)
